In [4]:
import pandas as pd
import os
from dotenv import load_dotenv
from langchain.docstore.document import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# 0. .env 파일에서 API 키 로드
load_dotenv()
# .env 파일에 OPENAI_API_KEY가 설정되어 있는지 확인
if os.getenv("OPENAI_API_KEY") is None:
    print("오류: .env 파일에 OPENAI_API_KEY를 설정해주세요.")
    exit()

# 1. 데이터 로드 및 전처리 (사용자께서 작성하신 코드와 동일)
try:
    df = pd.read_csv('db_drug_interactions.csv')
    print("CSV 파일 로드 성공!")
    print(f"데이터 수: {len(df)}개")

    documents = []
    for _, row in df.iterrows():
        content = f"Interaction between {row['Drug 1']} and {row['Drug 2']}: {row['Interaction Description']}"
        doc = Document(
            page_content=content,
            metadata={
                'drug1': row['Drug 1'],
                'drug2': row['Drug 2']
            }
        )
        documents.append(doc)
    print("Document 객체 변환 완료!")

except FileNotFoundError:
    print("오류: 'db_drug_interactions.csv' 파일을 찾을 수 없습니다. 파일이 코드와 같은 디렉토리에 있는지 확인하세요.")
    exit()

CSV 파일 로드 성공!
데이터 수: 191541개
Document 객체 변환 완료!


In [ ]:

# 2. Retriever (검색기) 설정
# OpenAI 임베딩 모델 사용
print("임베딩 모델 로딩 및 벡터 변환 중...")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# FAISS 벡터 데이터베이스 생성
vector_store = FAISS.from_documents(documents, embeddings)
print("벡터 데이터베이스 생성 완료!")

# Retriever 생성
retriever = vector_store.as_retriever()

In [ ]:



# 3. Generator (생성기) 설정
# OpenAI 챗 모델 사용
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
print("언어 모델(LLM) 설정 완료!")

# 프롬프트 템플릿 정의
prompt_template = """
당신은 약물 상호작용 전문가입니다. 제공된 정보를 바탕으로 사용자의 질문에 대해 명확하고 간결하게 한국어로 답변해주세요.
정보를 찾을 수 없다면, "제공된 정보 내에서는 해당 내용을 찾을 수 없습니다."라고 답변하세요.

[제공된 정보]
{context}

[질문]
{question}

[답변]
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


임베딩 모델 로딩 및 벡터 변환 중...


In [5]:

# 4. RetrievalQA 체인 생성 및 실행
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)

def ask_question(query):
    print(f"\n--- 질문: {query} ---")
    result = qa_chain.invoke(query)

    print("\n[생성된 답변]")
    print(result['result'])

    print("\n[답변 근거 문서]")
    for doc in result['source_documents']:
        print(f"- {doc.page_content}")

# --- 시스템 실행 ---
if __name__ == "__main__":
    ask_question("What is the interaction between Trioxsalen and Verteporfin?")
    ask_question("Paclitaxel과 Verteporfin의 상호작용은 무엇인가요?")

NameError: name 'retriever' is not defined